In [1]:
from imports import *
import prepare

In [2]:
## Getting data
df = pd.read_csv('data_science.csv')

In [3]:
## Initial cleaning
df = prepare.prep_data(df)

In [4]:
## Text cleaning
df[['clean', 'stemmed', 'lemmatized']] = prepare.prep_text(df, 'requirements')

In [5]:
df.head()

,company,location,mode,type,level,role,requirements,edu_bachelor,edu_master,edu_phd,edu_other,skills,associate,entry,mid_senior,clean,stemmed,lemmatized
0,PwC,"Austin, TX",Remote,Full time,Entry,Data Science & Machine Learning SA w/ Conversational AI,bachelor degree\nminimum years of experience: 4 year(s)\ndemonstrates thorough abiliti...,22,44,11,23,Python (Programming Language)\nMachine Learning\nSQL\nData Science\nDeep Learning\nDat...,0,1,0,bachelor degree minimum years experience 4 years demonstrates thorough abilities andor...,bachelor degre minimum year experi 4 year demonstr thorough abil andor proven record s...,bachelor degree minimum year experience 4 year demonstrates thorough ability andor pro...
1,Mastersworks,New York City Metropolitan Area,On-site,Full time,Mid-Senior,"VP, Data Science",identify and execute on predictive models to help internal teams at masterworks unders...,23,40,28,9,SQL\nArtificial Intelligence (AI)\nJavaScript\nEntrepreneurship\nAgile Methodologies\n...,0,0,1,identify execute predictive models help internal teams masterworks understand artists ...,identifi execut predict model help intern team masterwork understand artist market seg...,identify execute predictive model help internal team masterworks understand artist mar...
2,ReviveMed,"Cambridge, MA",Hybrid,Full time,Mid-Senior,Biological Data Science,"self-motivated, highly disciplined, and passionate about discovering the right therape...",10,55,31,4,SQL\nJavaScript\nMachine Learning\nLeadership\nPython (Programming Language)\nProgramm...,0,0,1,selfmotivated highly disciplined passionate discovering right therapeutics right patie...,selfmotiv highli disciplin passion discov right therapeut right patient phd ms comput ...,selfmotivated highly disciplined passionate discovering right therapeutic right patien...
3,IBM,"Baton Rouge, LA",On-site,Full time,Entry,Data Engineer,"2 years + experience with python, java, or other object-oriented programming languages...",0,100,0,0,SQL\nGitHub\nLeadership\nOracle Database\nGoogle Cloud Platform (GCP)\nApache Phoenix\...,0,1,0,2 years experience python java objectoriented programming languages handson experience...,2 year experi python java objectori program languag handson experi understand objector...,2 year experience python java objectoriented programming language handson experience u...
4,Expedia Group,"Austin, TX",Hybrid,Full time,Entry,"Data Scientist II, Product Analytics",2+ years of work experience doing quantitative analysis to tackle business problems\ns...,13,72,7,8,SQL\nJavaScript\nLeadership\nMicrosoft SQL Server\nCascading Style Sheets (CSS)\nC#\nO...,0,1,0,2 years work experience quantitative analysis tackle business problems strong analytic...,2 year work experi quantit analysi tackl busi problem strong analyt skill includ abil ...,2 year work experience quantitative analysis tackle business problem strong analytical...


# Exploration